<a href="https://colab.research.google.com/github/dine1717/END2/blob/Session5/S5_Assignmnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load required packages
import pandas as pd
import numpy as np

import os
import time
import pickle
import random
from statistics import mean,median,mode
import torch, torchtext
from torchtext.legacy import data


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Unzip Data**

In [ ]:
!unzip /content/drive/MyDrive/stanfordSentimentTreebank.zip

Archive:  /content/drive/MyDrive/stanfordSentimentTreebank.zip
   creating: stanfordSentimentTreebank/
  inflating: stanfordSentimentTreebank/datasetSentences.txt  
   creating: __MACOSX/
   creating: __MACOSX/stanfordSentimentTreebank/
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSentences.txt  
  inflating: stanfordSentimentTreebank/datasetSplit.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSplit.txt  
  inflating: stanfordSentimentTreebank/dictionary.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._dictionary.txt  
  inflating: stanfordSentimentTreebank/original_rt_snippets.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._original_rt_snippets.txt  
  inflating: stanfordSentimentTreebank/README.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._README.txt  
  inflating: stanfordSentimentTreebank/sentiment_labels.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._sentiment_labels.txt  
  inflating: stanfordSentimentTreebank/SOStr.tx

In [ ]:
dataset_sentences = pd.read_csv('stanfordSentimentTreebank/datasetSentences.txt', sep='\t')
print(f'{dataset_sentences.shape}   {dataset_sentences.head()}')
sentiment_labels = pd.read_csv('stanfordSentimentTreebank/sentiment_labels.txt', sep='|')                
sentiment_labels = sentiment_labels.rename(columns={'phrase ids': 'phrase_id', 
                                                    'sentiment values': 'sentiment_value'})
print(f'{sentiment_labels.shape}   {sentiment_labels.head()}')

(11855, 2)      sentence_index                                           sentence
0               1  The Rock is destined to be the 21st Century 's...
1               2  The gorgeously elaborate continuation of `` Th...
2               3                     Effective but too-tepid biopic
3               4  If you sometimes like to go to the movies to h...
4               5  Emerges as something rare , an issue movie tha...
(239232, 2)      phrase_id  sentiment_value
0          0          0.50000
1          1          0.50000
2          2          0.44444
3          3          0.50000
4          4          0.42708


In [ ]:
dataset_sentences.head()

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."


In [ ]:
dataset_sentences.shape

(11855, 2)

In [ ]:
sentiment_labels.head()

,phrase_id,sentiment_value
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


In [ ]:
sentiment_labels.shape

(239232, 2)

**Mappping Sentiments from Phrases to Sentences**

In [ ]:
sentence_sentiment = dataset_sentences
phrase_sentiment = dict(zip(list(sentiment_labels.phrase_id), 
                            list(sentiment_labels.sentiment_value)))

In [ ]:
# average sentiments per sentence (otherwise long sentences get high sentiments)
sentiments = [mean(phrase_sentiment[int(phrase_id)] for phrase_id in phrase_ids) 
              for phrase_ids in stree]
# define neutral sentiment as values within [0.5, 0.55) 
sentiments = [2 if sentiment >= 0.5 and sentiment < 0.55 else sentiment 
              for sentiment in sentiments]
# all negative sentiment values to 0, all positive sentiment values to 1
sentiments = [int(round(sentiment)) 
              for sentiment in sentiments]
# concatenate sentence sentiment values with corresponding sentence texts
sentence_sentiment['sentiment_value'] = sentiments

In [ ]:
min(sentence_sentiment['sentiment_value']), max((sentence_sentiment['sentiment_value']))

(0.420832, 0.6521161904761905)

In [ ]:
sentence_sentiment['label'] = sentence_sentiment.apply(lambda row: score_to_label(row['sentiment_value']), axis=1)

In [ ]:
print((100 * sentence_sentiment.sentiment_value.value_counts(normalize=True)).map('{:0.2f}%'.format))

0.581417    494
0.492492    490
0.522222    484
0.532828    467
0.598764    465
0.543906    458
0.489018    451
0.484331    450
0.616666    442
0.462738    430
0.619565    428
0.491358    424
0.465721    402
0.652116    401
0.652046    401
0.643791    377
0.439909    359
0.622221    354
0.449347    331
0.455975    330
0.482322    319
0.519229    310
0.449496    310
0.462964    248
0.454919    215
0.450095    214
0.444444    209
0.451941    205
0.450855    186
0.452383    182
0.443616    155
0.440419    126
0.420832    114
0.453443    101
0.470001     84
0.463471     81
0.470058     51
0.468107     49
0.500000     48
0.475035     44
0.451635     33
0.455824     32
0.464241     28
0.476592     22
0.479244     14
0.489474     11
0.493128      9
0.484469      5
0.489899      4
0.491750      4
0.502428      3
0.479229      1
Name: sentiment_value, dtype: int64


In [ ]:
# rename and select columns

sentence_sentiment = sentence_sentiment.rename(columns={'sentence': 'tweets', 'sentiment_value': 'labels'})
sentence_sentiment = sentence_sentiment[['tweets', 'labels']]
sentence_sentiment.sample(5)

,tweets,labels
7842,Nothing more substantial than a fitfully cleve...,1
2955,Last Orders nurtures the multi-layers of its c...,0
7911,The film seems a dead weight .,2
3884,The performances are all solid ; it merely lac...,2
2922,"... gripping and handsome execution , -LRB- bu...",0


**Defining Fields**

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequential to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐case.

In [ ]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
Tweet = data.Field(sequential=True, tokenize='spacy', batch_first=True, include_lengths=True)
Label = data.LabelField(tokenize='spacy', is_target=True, batch_first=True, sequential=False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:


In [ ]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.


In [ ]:
# Creating dataset
example = [data.Example.fromlist([sentence_sentiment.tweets[i],sentence_sentiment.labels[i]], fields) for i in range(sentence_sentiment.shape[0])] 

In [ ]:
## Save 
torch.save(example, 'example_v1.pt')

In [ ]:
sentimentDataset = data.Dataset(example, fields)

##Splitting Datasets

we can split into training, testing, and validation sets by using the split() method:

In [ ]:
(train, valid, test) = sentimentDataset.split(split_ratio=[0.6, 0.2, 0.2], random_state=random.seed(SEED))

In [ ]:
len(train) ,len(test) ,len(valid)

(7113, 2371, 2371)

In [ ]:
vars(train.examples[1])

{'labels': 1,
 'tweets': ['A',
  'confluence',
  'of',
  'kiddie',
  'entertainment',
  ',',
  'sophisticated',
  'wit',
  'and',
  'symbolic',
  'graphic',
  'design',
  '.']}

Data Augmentation

In [ ]:
# import libraries and prepare sentence text

import random
# for back translation
!pip install google_trans_new
import google_trans_new
from google_trans_new import google_translator
# for word tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Back Translation

In [ ]:
#translate a sentence to a random language,
# and translate back to original language

def back_translate(sentence, p=0.1):
  # do nothing with probability of 1-p
  if random.uniform(0,1) > p:
    return sentence

  # combine tokenized sentence into one string
  sentence = ' '.join(sentence)

  # instantiate translator
  translator = google_translator()

  # choose a target language
  available_langs = list(google_trans_new.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  #print(f"Translating to {google_trans_new.LANGUAGES[trans_lang]}")

  # translate to the target language
  translations = translator.translate(sentence, lang_tgt=trans_lang) 
  #print(translations)

  # translate back to original language
  translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
  #print(translations_en_random)

  # select only one translation
  if len(translations_en_random) > 1:
    translations_en_random = translations_en_random[0]

  return word_tokenize(translations_en_random)

Random Deletion

In [ ]:
#randomly delete words from a sentence with a given probability

def random_deletion(sentence, p=0.5): 
    # return if single word
    if len(sentence) == 1: 
        return sentence
    # delete words
    remaining = list(filter(lambda x: random.uniform(0,1) > p, sentence)) 
    # if nothing left, sample a random word
    if len(remaining) == 0: 
        return [random.choice(sentence)] 
    else:
        return remaining

Random Swap

In [ ]:
# randomly swap a pair of words in a sentence for a given # of times

def random_swap(sentence, n=5): 
    if len(sentence) < 2:
      return sentence
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

Data Augmentation Part

In [ ]:
for example in train.examples: 
  example.tweets = back_translate(example.tweets, p=0.01)
  example.tweets = random_deletion(example.tweets, p=0.1)
  example.tweets = random_swap(example.tweets, n=1)

In [ ]:
Building Vocabulary


## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabulary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all.
Let’s limit the vocabulary to a maximum of 10,000 words in our training set:

In [ ]:
MAX_VOCAB_SIZE = 10_000

Tweet.build_vocab(train, max_size = MAX_VOCAB_SIZE)
Label.build_vocab(train)

By default, torchtext will add two more special tokens

In [ ]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  10002
Size of label vocab :  3
Top 10 words appreared repeatedly : [('.', 5969), (',', 5273), ('the', 4519), ('and', 3323), ('of', 3244), ('a', 3208), ('to', 2260), ('-', 2036), ("'s", 1905), ('is', 1866)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})



## Making batches

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.
But at first declare the device we are using.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size = 32, 
    sort_key = lambda x: len(x.tweets),
    sort_within_batch=True, 
    device = device)

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

Build Model


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                               hidden_dim, 
                               num_layers=n_layers, 
                               dropout=dropout,
                               batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function (softmax)
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

print(model)

classifier(
  (embedding): Embedding(10002, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)


In [ ]:
# No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,242,503 trainable parameters


Train Evaluate Test 

In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc

Training Loop

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Evaluation Loop

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')


	Train Loss: 1.011 | Train Acc: 54.26%
	 Val. Loss: 0.989 |  Val. Acc: 56.25% 

	Train Loss: 0.977 | Train Acc: 56.63%
	 Val. Loss: 0.976 |  Val. Acc: 56.38% 

	Train Loss: 0.900 | Train Acc: 63.01%
	 Val. Loss: 0.842 |  Val. Acc: 70.46% 

	Train Loss: 0.806 | Train Acc: 74.38%
	 Val. Loss: 0.806 |  Val. Acc: 74.33% 

	Train Loss: 0.779 | Train Acc: 76.91%
	 Val. Loss: 0.821 |  Val. Acc: 72.29% 

	Train Loss: 0.762 | Train Acc: 78.30%
	 Val. Loss: 0.800 |  Val. Acc: 74.21% 

	Train Loss: 0.751 | Train Acc: 79.52%
	 Val. Loss: 0.797 |  Val. Acc: 73.67% 

	Train Loss: 0.737 | Train Acc: 81.86%
	 Val. Loss: 0.771 |  Val. Acc: 78.00% 

	Train Loss: 0.723 | Train Acc: 83.42%
	 Val. Loss: 0.763 |  Val. Acc: 78.96% 

	Train Loss: 0.713 | Train Acc: 84.28%
	 Val. Loss: 0.751 |  Val. Acc: 80.04% 



In [ ]:
# test model using testing dataset

model.load_state_dict(torch.load('saved_weights.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

<All keys matched successfully>

Test Loss: 0.750 | Test Acc: 80.42%


In [ ]:
# load weights and tokenizer
path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

# inference 
import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

<All keys matched successfully>

classifier(
  (embedding): Embedding(10002, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Positive'